In [1]:
from pyspark.sql import SparkSession

In [10]:
from pyspark.sql.functions import col,mean

In [3]:
spark = SparkSession.builder.appName("handleMissing").getOrCreate()

SLF4J: Class path contains multiple SLF4J bindings.
SLF4J: Found binding in [jar:file:/home/glue_user/spark/jars/log4j-slf4j-impl-2.17.2.jar!/org/slf4j/impl/StaticLoggerBinder.class]
SLF4J: Found binding in [jar:file:/home/glue_user/spark/jars/slf4j-reload4j-1.7.36.jar!/org/slf4j/impl/StaticLoggerBinder.class]
SLF4J: Found binding in [jar:file:/home/glue_user/aws-glue-libs/jars/log4j-slf4j-impl-2.17.2.jar!/org/slf4j/impl/StaticLoggerBinder.class]
SLF4J: Found binding in [jar:file:/home/glue_user/aws-glue-libs/jars/slf4j-reload4j-1.7.36.jar!/org/slf4j/impl/StaticLoggerBinder.class]
SLF4J: See http://www.slf4j.org/codes.html#multiple_bindings for an explanation.
SLF4J: Actual binding is of type [org.apache.logging.slf4j.Log4jLoggerFactory]
Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
25/03/18 16:31:10 WARN Utils: Service 'SparkUI' could not bind on port 4040. Attempting port 4041.


In [4]:
# Sample Data with Null Values
data = [
    (1, "Alice", 50000, "HR"),
    (2, "Bob", None, "IT"),
    (3, "Charlie", 70000, None),
    (4, "David", None, "Finance"),
    (5, None, 90000, "Marketing"),
    (6, "Eve", 100000, None)
]

columns = ["emp_id", "name", "salary", "department"]

# Create DataFrame
df = spark.createDataFrame(data, columns)

In [5]:
df.printSchema()

root
 |-- emp_id: long (nullable = true)
 |-- name: string (nullable = true)
 |-- salary: long (nullable = true)
 |-- department: string (nullable = true)



In [6]:
df.show()

+------+-------+------+----------+
|emp_id|   name|salary|department|
+------+-------+------+----------+
|     1|  Alice| 50000|        HR|
|     2|    Bob|  null|        IT|
|     3|Charlie| 70000|      null|
|     4|  David|  null|   Finance|
|     5|   null| 90000| Marketing|
|     6|    Eve|100000|      null|
+------+-------+------+----------+



In [7]:
# Write a PySpark query to drop rows where any column has a NULL value.
df_no_nulls = df.na.drop()
df_no_nulls.show()

+------+-----+------+----------+
|emp_id| name|salary|department|
+------+-----+------+----------+
|     1|Alice| 50000|        HR|
+------+-----+------+----------+



In [8]:
df.show()

+------+-------+------+----------+
|emp_id|   name|salary|department|
+------+-------+------+----------+
|     1|  Alice| 50000|        HR|
|     2|    Bob|  null|        IT|
|     3|Charlie| 70000|      null|
|     4|  David|  null|   Finance|
|     5|   null| 90000| Marketing|
|     6|    Eve|100000|      null|
+------+-------+------+----------+



In [19]:
#Replace missing salary values with the average salary.
avgSal = df.select(mean(col("salary"))).collect()[0][0]

In [20]:
print(avgSal)

77500.0


Replace missing salary values with the average salary.
Replace missing department values with "Unknown".

In [22]:
df_filled = df.na.fill({"salary":avgSal,"department":"unknown"})

In [23]:
df_filled.show()

+------+-------+------+----------+
|emp_id|   name|salary|department|
+------+-------+------+----------+
|     1|  Alice| 50000|        HR|
|     2|    Bob| 77500|        IT|
|     3|Charlie| 70000|   unknown|
|     4|  David| 77500|   Finance|
|     5|   null| 90000| Marketing|
|     6|    Eve|100000|   unknown|
+------+-------+------+----------+



In [26]:
from pyspark.sql.functions import col

In [28]:
#Retrieve only the rows where salary is NULL
df.filter(col("salary").isNull()).show()

+------+-----+------+----------+
|emp_id| name|salary|department|
+------+-----+------+----------+
|     2|  Bob|  null|        IT|
|     4|David|  null|   Finance|
+------+-----+------+----------+



In [ ]:
#Write a PySpark query to drop rows where at least two columns have NULL values.
